# Running Dask on AzureML


In [1]:
from azureml.core import Workspace, Experiment
from azureml.widgets import RunDetails
from azureml.core.runconfig import RunConfiguration, MpiConfiguration
from azureml.train.estimator import Estimator

## Starting the cluster

In [2]:
ws = Workspace.from_config()
ws

Workspace.create(name='ncus-azureml', subscription_id='6560575d-fa06-4e7d-95fb-f962e74efd7a', resource_group='copetersrg')

In [3]:
ct = ws.compute_targets['dask-cluster']
ct

AmlCompute(workspace=Workspace.create(name='ncus-azureml', subscription_id='6560575d-fa06-4e7d-95fb-f962e74efd7a', resource_group='copetersrg'), name=dask-cluster, id=/subscriptions/6560575d-fa06-4e7d-95fb-f962e74efd7a/resourceGroups/copetersrg/providers/Microsoft.MachineLearningServices/workspaces/ncus-azureml/computes/dask-cluster, type=AmlCompute, provisioning_state=Succeeded, location=northcentralus, tags=None)

Starting the Dask cluster using an Estimator with MpiConfiguration. Make sure the cluster is able to scale up to 10 nodes or change the `node_count` below. 

In [4]:
est = Estimator('dask', 
                compute_target=ct, 
                entry_script='startDask.py', 
                conda_dependencies_file='environment.yml', 
                script_params={'--datastore': ws.get_default_datastore()},
                node_count=40,
                distributed_training=MpiConfiguration())

run = Experiment(ws, 'dask').submit(est)

In [5]:
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'NOTSET',…

In [8]:
import time
from IPython.display import clear_output

print("waiting for scheduler node's ip")
while run.get_status() != 'Canceled' and 'headnode' not in run.get_metrics():
    print('.', end ="")
    time.sleep(5)

clear_output()

if run.get_status() == 'Canceled':
    print('Run was canceled')
else:
    headnode = run.get_metrics()['headnode']
    print(run.get_metrics())

{'headnode': '10.1.0.5', 'cluster': 'scheduler: 10.1.0.5:8786, dashboard: 10.1.0.5:8787', 'datastore': '/mnt/batch/tasks/shared/LS_root/jobs/ncus-azureml/azureml/dask_1576287903_6e3e23ff/mounts/workspaceblobstore'}


In [48]:
import dask
from dask.distributed import Client

c = Client(f'tcp://{headnode}:8786')
c.restart()
c

Client Scheduler: tcp://10.1.0.5:8786 Dashboard: http://10.1.0.5:8787/status,Cluster Workers: 40 Cores: 320 Memory: 2.36 TB


## Establish the port-forwarding from Compute Instance to Dask Dashboard

In [49]:
print(f'ssh daskuser@{headnode} -L 8788:{headnode}:8787')

ssh daskuser@10.1.0.5 -L 8788:10.1.0.5:8787


Make sure to leave the terminal tab open to keep the port-forward running

## Run some jobs on the cluster
If you are able to see the Bokeh app, it is time to use the cluster. Thanks to the port forward, the scheduler appears to the notebook VM at `tcp://localhost:8786`. You should see 10 workers.

In [50]:
import dask
import dask.dataframe as dd

from azureml.opendatasets import NoaaIsdWeather

from datetime import datetime, timedelta

In [51]:
start = datetime(2010, 1, 1)

In [55]:
fns = [dask.delayed(NoaaIsdWeather(start+timedelta(days=i), start+timedelta(days=i+1)).to_pandas_dataframe) for i in range(365*10)]

In [57]:
sample_df = NoaaIsdWeather(start, start+timedelta(days=1)).to_pandas_dataframe()
sample_df.head()

Target paths: ['/year=2010/month=1/']
Looking for parquet files...
Reading them into Pandas dataframe...
Reading ISDWeather/year=2010/month=1/part-00009-tid-2760641154746282667-d6a50598-0478-455f-9217-ce4afb96a9bb-36.c000.snappy.parquet under container isdweatherdatacontainer
Done.


,usaf,wban,datetime,latitude,longitude,elevation,windAngle,windSpeed,temperature,seaLvlPressure,...,pastWeatherIndicator,precipTime,precipDepth,snowDepth,stationName,countryOrRegion,p_k,year,day,version
0,999999,63858,2010-01-01 00:00:00,32.457,-87.242,59.0,0.0,0.0,6.2,NaN,...,NaN,1.0,0.0,NaN,SELMA 13 WNW,US,999999-63858,2010,1,1.0
1,999999,63858,2010-01-01 00:05:00,32.457,-87.242,59.0,NaN,NaN,6.3,NaN,...,NaN,NaN,NaN,NaN,SELMA 13 WNW,US,999999-63858,2010,1,1.0
2,999999,63858,2010-01-01 00:10:00,32.457,-87.242,59.0,NaN,NaN,6.3,NaN,...,NaN,NaN,NaN,NaN,SELMA 13 WNW,US,999999-63858,2010,1,1.0
3,999999,63858,2010-01-01 00:15:00,32.457,-87.242,59.0,NaN,NaN,6.4,NaN,...,NaN,NaN,NaN,NaN,SELMA 13 WNW,US,999999-63858,2010,1,1.0
4,999999,63858,2010-01-01 00:20:00,32.457,-87.242,59.0,NaN,NaN,5.6,NaN,...,NaN,NaN,NaN,NaN,SELMA 13 WNW,US,999999-63858,2010,1,1.0


In [59]:
fns[0].compute()

KilledWorker: ('to_pandas_dataframe-f8ac14f6-5cac-4fe6-9745-16e7676c469c', <Worker 'tcp://10.1.0.38:46861', memory: 0, processing: 1>)

In [58]:
df = dd.from_delayed(fns)

KilledWorker: ('to_pandas_dataframe-f8ac14f6-5cac-4fe6-9745-16e7676c469c', <Worker 'tcp://10.1.0.34:33429', memory: 0, processing: 1>)

In [26]:
c

Client Scheduler: tcp://10.1.0.5:8786 Dashboard: http://10.1.0.5:8787/status,Cluster Workers: 40 Cores: 320 Memory: 2.36 TB


In [24]:
help(dset._dataflow)

Help on Dataflow in module azureml.dataprep.api.dataflow object:

class Dataflow(builtins.object)
 |  A Dataflow represents a series of lazily-evaluated, immutable operations on data.
 |      It is only an execution plan. No data is loaded from the source until you get data from the Dataflow using one of `head`, `to_pandas_dataframe`, `get_profile` or the write methods.
 |  
 |  .. remarks::
 |  
 |      Dataflows are usually created by supplying a data source. Once the data source has been provided, operations
 |          can be added by invoking the different transformation methods available on this class. The result of adding
 |          an operation to a Dataflow is always a new Dataflow.
 |  
 |      The actual loading of the data and execution of the transformations is delayed as much as possible and will not
 |          occur until a 'pull' takes place. A pull is the action of reading data from a Dataflow, whether by asking to
 |          look at the first N records in it or by 

In [ ]:
exp = Experiment(ws, 'dask')
exp

In [ ]:
runs = exp.get_runs()
run = next(runs)
run

In [ ]:
import dask
import dask.dataframe as dd

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

from datetime import datetime

%matplotlib inline

In [ ]:
from dask.distributed import Client

c = Client('tcp://localhost:8786')
c.restart()
c

In [ ]:
ds = run.get_metrics()['datastore']
ds

In [ ]:
path = ds + ''
path

In [ ]:
def load_data(path):
    df = dd.read_csv(path+'/datasets/isd/*data.csv', dtype={'usaf': 'object'})
    return df

In [ ]:
NoaaIsd().to_dask_dataframe()

In [ ]:
df = dask.delayed(load_data)(path).compute()

In [ ]:
df.head()

In [ ]:
df.npartitions

In [ ]:
%time len(df)

In [ ]:
df.datetime = dd.to_datetime(df.datetime).dt.floor('d')

In [ ]:
df = df.repartition(npartitions=150)

In [ ]:
df = df.set_index(df.datetime, sorted=True).persist()

In [ ]:
df.head()

In [ ]:
%time len(df)

In [ ]:
df.npartitions

In [ ]:
df2 = df.persist()

In [ ]:
df.describe().compute()

In [ ]:
means = df.groupby(df.index).mean().compute()
means.head()

In [ ]:
df = df.drop(['datetime'], axis=1)

In [ ]:
df.index

In [ ]:
def write_data(path):
    df.to_parquet(path)

In [ ]:
a = dask.delayed(write_data)(ds+'/dask/outputs/isd').compute()

In [ ]:
counts = df.groupby([df.index.month, df.index.year]).day.count().compute()

In [ ]:
cs = [counts[month][2015] for month in range(1, 13)]
cs

In [ ]:
for col in list(means.columns):
    fig = plt.figure(figsize=(16, 8))
    plt.style.use('dark_background')
    means[col].plot(color='b')
    plt.title('Average of {}'.format(col))
    plt.xlim([datetime(2015, 1, 1), datetime(2015, 12, 1)])
    plt.grid()
    
    run.log_image(col, plot=plt)

In [ ]:
df.memory_usage(index=True, deep=True).sum().compute()

In [ ]:
df.info()

See if the cluster works

In [ ]:
import time
import numpy as np
from dask import delayed, visualize

def inc(x):
    time.sleep(abs(np.random.normal(5, 2)))
    return x + 1

fut = []
for i in range(10):
    fut.append( c.submit(delayed(inc), i) )

fut

In [ ]:
for i in fut:
    print(i.result())

In [ ]:
def sum(a):
    x = 0
    for y in a:
        x += y
    return x

results = []
for f in fut:
    results.append(f.result())
    
fut2 = c.submit(sum, results)
fut2

In [ ]:
fut2.result().compute()

In [ ]:
visualize(fut2.result())

# Training on Large Datasets
(from https://github.com/dask/dask-tutorial)

Sometimes you'll want to train on a larger than memory dataset. `dask-ml` has implemented estimators that work well on dask arrays and dataframes that may be larger than your machine's RAM.

In [ ]:
from dask.distributed import Client
import joblib
import dask.array as da
import dask.delayed
from sklearn.datasets import make_blobs
import numpy as np

We'll make a small (random) dataset locally using scikit-learn.

In [ ]:
n_centers = 12
n_features = 20

X_small, y_small = make_blobs(n_samples=1000, centers=n_centers, n_features=n_features, random_state=0)

centers = np.zeros((n_centers, n_features))

for i in range(n_centers):
    centers[i] = X_small[y_small == i].mean(0)
    
centers[:4]

The small dataset will be the template for our large random dataset.
We'll use `dask.delayed` to adapt `sklearn.datasets.make_blobs`, so that the actual dataset is being generated on our workers. 

In [ ]:
n_samples_per_block = 200000
n_blocks = 500

delayeds = [dask.delayed(make_blobs)(n_samples=n_samples_per_block,
                                     centers=centers,
                                     n_features=n_features,
                                     random_state=i)[0]
            for i in range(n_blocks)]
arrays = [da.from_delayed(obj, shape=(n_samples_per_block, n_features), dtype='float64')
          for obj in delayeds]
X = da.concatenate(arrays)
X

In [ ]:
# Check the size of the array
X.nbytes / 1e9

In [ ]:
# Only run this on the cluster.
X = X.persist()  

The algorithms implemented in Dask-ML are scalable. They handle larger-than-memory datasets just fine.

They follow the scikit-learn API, so if you're familiar with scikit-learn, you'll feel at home with Dask-ML.

In [ ]:
from dask_ml.cluster import KMeans
clf = KMeans(init_max_iter=3, oversampling_factor=10)

In [ ]:
%time clf.fit(X)

In [ ]:
clf.labels_

In [ ]:
clf.labels_[:10].compute()

## Shut cluster down
To shut the cluster down, cancel the job that runs the cluster. 

In [ ]:
for run in ws.experiments['dask'].get_runs():
    if run.get_status() == "Running":
        print(f'cancelling run {run.id}')
        run.cancel()

### Just for convenience, get the latest running Run

In [ ]:
for run in ws.experiments['dask'].get_runs():
    if run.get_status() == "Running":
        print(f'latest running run is {run.id}')
        break